#### Try to convert input sentences into numeric sequences

In [177]:
import tensorflow as tf

Korpus merupakan kumpulan teks yang tersimpan secara elektronik untuk berbagai kebutuhan penyelidikan dan penelitian, salah satunya dalam pemrosesan bahasa alami (Natural Language Processing).

Dalam konteks pemrosesan bahasa alami (Natural Language Processing), "corpus" merujuk pada kumpulan teks yang digunakan untuk analisis atau pelatihan model. Corpus bisa terdiri dari berbagai jenis teks, seperti artikel, buku, percakapan, atau dokumen lainnya. 

Berikut adalah beberapa poin penting tentang corpus:

- Sumber Data: Corpus berfungsi sebagai sumber data untuk melatih model bahasa, membantu model memahami pola dan struktur bahasa.
- Ukuran: Corpus bisa bervariasi dalam ukuran, dari beberapa kalimat hingga jutaan kata, tergantung pada tujuan analisis.
Kualitas: Kualitas corpus sangat penting; corpus yang baik harus representatif dan relevan dengan tugas yang ingin diselesaikan.
- Penggunaan: Dalam pelatihan model, corpus digunakan untuk mengajarkan model tentang kosakata, tata bahasa, dan konteks penggunaan kata.

In previous lab, i use the TextVectorization layer to build vocabulary from my corpus. Itu menghasilkan daftar di mana kata-kata yang lebih sering muncul memiliki indeks yang lebih rendah.

In [178]:
#Sample inputs

sentences=[
    'Ayu cantik',
    'Wahyuni bukan Wahyu',
    'I love me',
    'Am I supernova huh?'
]

#inisialisasi layer
vectorize_layer=tf.keras.layers.TextVectorization()

#Build vocabulary
#tiap kata pada sentences disimpan dalam bentuk indeks
vectorize_layer.adapt(sentences)

#Get the vocabulary
vocabulary=vectorize_layer.get_vocabulary()

#Print the token index
for index, word in enumerate(vocabulary):
    print(index, word)



0 
1 [UNK]
2 i
3 wahyuni
4 wahyu
5 supernova
6 me
7 love
8 huh
9 cantik
10 bukan
11 ayu
12 am


In [179]:
#Memeriksa token pada suatu kalimat
#convert sentence to integer sequences

sample_sentence=[
        'saya makan nasi',
        'i love me',
        'i love nasi padang'
]

sample_sentence2='am i supernova huh? '

#convert
sequences1=vectorize_layer(sample_sentence)
sequences2=vectorize_layer(sample_sentence2)

# for index, word in enumerate(sample_sentence):
print(sequences1)
print("\n",sequences2)




tf.Tensor(
[[1 1 1 0]
 [2 7 6 0]
 [2 7 1 1]], shape=(3, 4), dtype=int64)

 tf.Tensor([12  2  5  8], shape=(4,), dtype=int64)


For a given list of string inputs (such as the 4-item `sentences` list above), you will need to apply the layer to each input. There's more than one way to do this. Let's first use the `map()` method and see the results.

Untuk list of string inputs tertentu (seperti daftar `sentences` berisi 4 item di atas), Anda perlu menerapkan layer tersebut ke setiap masukan. Ada lebih dari satu cara untuk melakukannya. Mari kita gunakan metode `map()` terlebih dahulu dan lihat hasilnya.

In [192]:
#convert the list to tf.data.Dataset
#membuat daftar dari kalimat menjadi suatu dataset
#setiap elemen dari dataset adalah satu kalimat dari sentences
sentences_dataset = tf.data.Dataset.from_tensor_slices(sentences)

#mapping function untuk convert each inputs to integer sequence
#aplikasikan vectorize_layer pada setiap sentence 
sequences_convert=sentences_dataset.map(vectorize_layer)

print(sequences_convert)



<_MapDataset element_spec=TensorSpec(shape=(None,), dtype=tf.int64, name=None)>


In [193]:
for sentencenya, sequencenya in zip(sentences, sequences_convert):
    print(f"{sentencenya} ----> {sequencenya}")


Ayu cantik ----> [11  9]
Wahyuni bukan Wahyu ----> [ 3 10  4]
I love me ----> [2 7 6]
Am I supernova huh? ----> [12  2  5  8]


## Padding


index for padding --> [0]

#### Post-padding

In [194]:
sequences_post = vectorize_layer(sentences)

print('Input: ')
print(sentences)
[print(sequence.numpy()) for sequence in sequences_convert]
print()



print('Output: ')
print(sequences_post)

Input: 
['Ayu cantik', 'Wahyuni bukan Wahyu', 'I love me', 'Am I supernova huh?']
[11  9]
[ 3 10  4]
[2 7 6]
[12  2  5  8]

Output: 
tf.Tensor(
[[11  9  0  0]
 [ 3 10  4  0]
 [ 2  7  6  0]
 [12  2  5  8]], shape=(4, 4), dtype=int64)


#### Pre-padding

In [182]:
sequences_pre=tf.keras.utils.pad_sequences(sequences_convert, padding='pre')

print('Input: ')
[print(sequence.numpy()) for sequence in sequences_convert]
print()

print('Output: ')
print(sequences_pre)

Input: 
[11  9]
[ 3 10  4]
[2 7 6]
[12  2  5  8]

Output: 
[[ 0  0 11  9]
 [ 0  3 10  4]
 [ 0  2  7  6]
 [12  2  5  8]]


#### Post-padding and Limit Size

truncating=memotong data jika kepanjangan

In [187]:
sequences_post_trunc=tf.keras.utils.pad_sequences(sequences_convert, maxlen=3, padding='post')

print('Input: ')
[print(sequence.numpy()) for sequence in sequences_convert]
print()

print('Output: ')
print(sequences_post_trunc)

Input: 
[11  9]
[ 3 10  4]
[2 7 6]
[12  2  5  8]

Output: 
[[11  9  0]
 [ 3 10  4]
 [ 2  7  6]
 [ 2  5  8]]


### Ragged Tensor

Cara lain untuk menyiapkan urutan untuk prepadding adalah dengan menyetel TextVectorization untuk menghasilkan tensor yang tidak rata. Ini berarti output tidak akan secara otomatis diberi post-padding.

Ragged tensor adalah jenis tensor yang memungkinkan setiap elemen memiliki panjang yang berbeda. Ini sangat berguna dalam pemrosesan teks karena kalimat memiliki jumlah kata yang berbeda-beda.

Kamu tidak perlu menambah nilai kosong, dan data tetap memiliki panjang yang berbeda, tetapi TensorFlow tetap bisa memprosesnya dengan cara yang lebih efisien daripada padding.

In [188]:
#membuat layer dengan outputnya ragged tensor
vectorize_rlayer = tf.keras.layers.TextVectorization(ragged=True)

vectorize_rlayer.adapt(sentences)

ragged_sequences=vectorize_rlayer(sentences)

print(ragged_sequences)

<tf.RaggedTensor [[11, 9], [3, 10, 4], [2, 7, 6], [12, 2, 5, 8]]>


.numpy() mengonversi tensor ragged menjadi array NumPy yang lebih sederhana, sehingga dapat diproses langsung oleh pad_sequences (menambahkan urutan/padding) yang berfungsi memastikan semua sequence memiliki panjang yang sama.

In [189]:
# Pre-pad the sequences in the ragged tensor
sequencesr_pre=tf.keras.utils.pad_sequences(ragged_sequences.numpy())

print(sequencesr_pre)


[[ 0  0 11  9]
 [ 0  3 10  4]
 [ 0  2  7  6]
 [12  2  5  8]]


Hasilnya adalah ragged tensor yang berisi urutan angka untuk setiap kalimat, dengan panjang yang sesuai dengan jumlah kata dalam kalimat tersebut.

## Out-of-vocabulary tokens (OOV)

oov --> [1]

Input kata yang tidak dikenali oleh vocabulary (oov) akan ditandai dengan index [1]. 

In [190]:
sentences_with_oov=[
    'i really love you, but',
    'we are lie',
    'I love the scent'
]

sequences_with_oov= vectorize_layer(sentences_with_oov)

for sentence, sequence in zip(sentences_with_oov, sequences_with_oov):
    print(f"{sentence} ----> {sequence}")

i really love you, but ----> [2 1 7 1 1]
we are lie ----> [1 1 1 0 0]
I love the scent ----> [2 7 1 1 0]
